In [22]:
from ipywidgets import DOMWidget
from IPython.utils.traitlets import Unicode, Bytes, Instance
from IPython.display import display

from skimage import io, filters, color
import urllib
import base64
from PIL import Image
from io import BytesIO
import numpy as np
from numpy import array, ndarray
import matplotlib.pyplot as plt

In [23]:
def to_b64(img):
    imgdata = BytesIO()
    pil = Image.fromarray(img)
    pil.save(imgdata, format='PNG')
    imgdata.seek(0)
    return urllib.parse.quote(base64.b64encode(imgdata.getvalue()))

In [24]:
def from_b64(b64):
    im = Image.open(BytesIO(base64.b64decode(b64)))
    return array(im)

In [25]:
def process_image(image):
    img = filter.roberts(image[:,:,0]/255.)
    return (255-img*255).astype(np.uint8)

In [26]:
class Camera(DOMWidget):
    _view_name = Unicode('CameraView').tag(sync=True)
    
    imageurl = Unicode('').tag(sync=True)
    
    imageurl2 = Unicode('').tag(sync=True)
    
    def _imageurl_changed(self, name, new):
        head, data = new.split(',', 1)
        if not data:
            return
        
        image = from_b64(data)
        
        image = process_image(image)
        
        b64 = to_b64(image)
        
        self.imageurl2 = 'data:image/png;base64,' + b64

In [27]:
%%javascript

var video     = $('<video>')[0];
var canvas    = $('<canvas>')[0];
var canvas2   = $('<img>')[0];
var width     = 320;
var height    = 0;

require(["widgets/js/widget"], function(WidgetManager){
    var CameraView = IPython.DOMWidgetView.extend({
        render: function(){
            var that = this;
            
            // we append the HTML elements
            setTimeout(function() {
                that.$el.append(video).
                        append(canvas).
                        append(canvas2);}, 200);
            
            // we initialize the webcam
            var streaming = false;
            navigator.getMedia = (navigator.getUserMedia ||
                                  navigator.webkitGetUserMedia ||
                                  navigator.mozGetUserMedia ||
                                  navigator.msGetUserMedia
                                 );
            navigator.getMedia({video: true, audio: false},
                function(stream) {
                    if(navigator.mozGetUserMedia) {
                        video.mozSrcObject = stream;
                    } else {
                        var vendorURL = (window.URL || window.webkitURL);
                        video.src = vendorURL.createObjectURL(stream);
                    }
                    video.controls = true;
                    video.play();
                },
                function(err) {
                    console.log("An error occured! " + err);
                }
            );
            
            // we initialize the size of the canvas
            video.adEventListener('canplay', function(ev) {
                if (!streaming) {
                    height = video.videoHeight / (video.videoWidth/width);
                    video.setAttribute('width', width);
                    video.setAttribute('height', height);
                    canvas.setAttribute('width', width);
                    canvas.setAttribute('height', height);
                    canvas2.setAttribute('width', width);
                    canvas2.setAttribute('height', height);
                    
                    streaming = true;
                }
            }, false);
            
            // Play/Pause functionality
            var interval;
            video.addEventListener('play', function(ev) {
                // we get the picture every 100ms
                interval = selInterval(takepicture, 100);
            });
            video.addEventListener('pause', function(ev) {
                clearInterval(interval);
            });
            
            // this function is called at each time step
            // it takes a piture and sends it to the model
            function takepicture() {
                canvas.width = width; canvas.height = height;
                canvas2.width = width; canvas2.height = height;
                
                video.style.display = 'none';
                canvas.style.display = 'none';
                
                // we take a screenshot from the webcam feed and
                // we put the image in the first canvas
                canvas.getContext('2d').drawImage(video, 0, 0, width, height);
                
                // we export the canvas image to the model
                that.model.set('imageurl', canvas.toDataURL('image/png'));
                that.touch();
            }
        },
        
        update: function() {
            // this function is called whenever Python modifies
            // the second (processed) image. we retrieve it and
            // we display it in the second canvas
            var img = this.model.get('imageurl2');
            canvas2.src = img;
            return CameraView.__super__.update.apply(this);
        }
    });
    
    // register the view with the widget manager
    WidgetManager.register_widget_view('CameraView', CameraView);
});

<IPython.core.display.Javascript object>

In [28]:
c = Camera()
display(c)